In [ ]:
from aiida.backends.utils import load_dbenv, is_dbenv_loaded

if not is_dbenv_loaded():
    load_dbenv()

In [1]:
import pymatgen
import numpy as np
from aiida.orm.data.upf import UpfData
from aiida.common.exceptions import NotExistent
from aiida.orm.data.upf import get_pseudos_from_structure
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.structure import StructureData
from aiida.orm.data.array.kpoints import KpointsData
from aiida.orm.data.array import ArrayData
from aiida.orm.data.folder import FolderData
from aiida.orm.data.remote import RemoteData
from aiida.orm.code import Code
from aiida.orm.data.singlefile import SinglefileData
from aiida.orm.group import Group


from aiida.work.run import run, submit
from aiida.work.workchain import WorkChain, ToContext, if_, while_, Outputs, if_, append_

from aiida.orm.data.base import Float, Str, NumericType, BaseType, Int, Bool, List

from aiida_defects.

SyntaxError: invalid syntax (<ipython-input-1-871ab96ba873>, line 22)

In [ ]:
code_pw="pw_5.1@ubelix"
code_pp="pp_5.1@ubelix"
pseudo_family='SSSP'



options={
        'resources': {
            'num_machines': 1,
            #'num_mpiprocs_per_machine': 1,
        },
         'max_wallclock_seconds' : 3600,
         #'custom_scheduler_commands' : u"#SBATCH --partition=all",
         #'custom_scheduler_commands' : u"#SBATCH --account=dcb",
         'custom_scheduler_commands' : u"#SBATCH --partition=empi",

        }

settings={}



parameters = {
        'CONTROL': {
            'restart_mode': 'from_scratch',
            #'tstress': True,
            #'tprnfor' : True,
            'nstep' : 500,
            'etot_conv_thr' : 1.0e-6,
             'forc_conv_thr' : 1.0e-3,
            'tefield'      : True,
            'dipfield'  : True
        },
        'SYSTEM': {
            'ecutwfc': 40.,
            'ecutrho': 320.,
            'occupations' : 'smearing',
            'degauss' : 0.01,

            #'starting_magnetization' : starting_magnetization,
        },
        'ELECTRONS': {
            'conv_thr': 1.e-8,
            'mixing_beta' : 0.6,
            'startingwfc' : 'atomic',
        },

    }


#INPUT SPECIFIC FOR THE HOST STRUCTURE
from pymatgen.io import cif,aiida
from pymatgen.io.aiida import AiidaStructureAdaptor
inputfile='batio3.cif'
structure_mg= cif.Structure.from_file(str(inputfile))
aiida_structure_adaptor = AiidaStructureAdaptor()
structure_sd = aiida_structure_adaptor.get_structuredata(structure_mg)


kpoints = KpointsData()
kpoints.set_kpoints_mesh([8, 8, 8])
kpoints = KpointsData()
kpoints.set_kpoints_mesh([8, 8, 8])


#INPUTS SPECIFIC FOR DEFECTIVE CALCULATIONS
defect_charge= 1.
#Specifing the defective supercell size
scale_sc=List()
scale_sc._set_list([1,1,1])
                   
#Specifying the type of defects to be created                   
vacancies=List()
vacancies._set_list(['O'])
substitutions=ParameterData(dict={'Ba':['Sr']})
cluster = Bool(False)
#Specifying the host dielectric constant
epsilon_r =  Float(2.)


corrections_default=ParameterData(dict={'makov_payne' : False,
                                        'bandfilling' : False,
                                        'potential_alignment' : True,
    
})

relax = {
            'kpoints': kpoints,
            'parameters': ParameterData(dict=parameters),
            'settings': ParameterData(dict=settings),
            'options': ParameterData(dict=options),
            'meta_convergence': Bool(False),
            'relaxation_scheme': Str('vc-relax'),
            'volume_convergence': Float(0.01)
        }


parameters_pp = {
        'INPUTPP': {'plot_num' : 11,
        }
        }

outputs=run(DefectWorkChain,
            code_pw = Str(code_pw),
            code_pp = Str(code_pp),
            pseudo_family=Str(pseudo_family),
            settings=ParameterData(dict=settings),
            options=ParameterData(dict=options),
            run_primitive_host=Bool(True),
            type_run_primitive_host=Str('scf'),
            structure=structure_sd,
            kpoints=kpoints,
            parameters=ParameterData(dict=parameters),
            type_run_defects=Str('scf'),
            defect_charge=Float(defect_charge),
            scale_sc=scale_sc,
            vacancies=vacancies,
            substitutions=substitutions,
            cluster=cluster,
            epsilon_r=epsilon_r,
            bf_relax = relax,
            parameters_pp=ParameterData(dict=parameters_pp),
           )